In [1]:
# from functions import get_current_location
# location = get_current_location()
# if location:
#     latitude, longitude = location
#     print(f'Current Latitude: {latitude}, Longitude: {longitude}')
# else:
#     print('Failed to determine current location.')

In [1]:
from route import Route
from api_route import ApiRoute

current_position = {'longitude': 130.841366,
                    'latitude': -12.45519}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 2000.0 # in meters
number_sites = 150

route = Route()
api_route = ApiRoute(route)

api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# route.find_closest_row(current_position)
# route.find_closest_rows(route.get_control_stops_data)
# route.insert_to_control_stops()

Nearest index in csv file: 63
Nearest index in dataframe: 61
      longitude   latitude  altitudeSmoothed  inclinationSmoothed  \
0    130.844803 -12.456034         11.138923            -0.008233   
1    131.002346 -12.478918         35.722314            -0.008522   
2    131.073674 -12.633291         36.143921             0.001992   
3    131.122370 -12.804436         38.293296             0.006307   
4    131.116680 -12.977022         86.681074            -0.008452   
..          ...        ...               ...                  ...   
145  138.246819 -34.297629          9.861291             0.000308   
146  138.355575 -34.452771         14.337920            -0.001938   
147  138.502880 -34.588772         13.666016            -0.001948   
148  138.589449 -34.752723         10.888153             0.001120   
149  138.600712 -34.928189         64.449372            -0.011632   

      cumDistance  maxSpeed       theta  timeAtMaxSpeed  cumTimeAtMaxSpeed  \
0    1.358064e+03        40   41

In [2]:
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser

api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser, print_is_requested=True)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved:
        name  longitude  latitude  altitude      UTC_offset
site_id                                                    
589020     0    130.841  -12.4552        32 0 days 09:30:00
589021     1    130.999  -12.4758        29 0 days 09:30:00
589022     2    131.073  -12.6294        26 0 days 09:30:00
589023     3    131.123  -12.8008        36 0 days 09:30:00
589024     4    131.116  -12.9737       102 0 days 09:30:00
...      ...        ...       ...       ...             ...
589165   145    138.275  -34.3310        15 0 days 10:30:00
589166   146    138.388  -34.4848        17 0 days 10:30:00
589167   147    138.524  -34.6259        14 0 days 10:30:00
589168   148    138.579  -34.7917        11 0 days 10:30:00
589169   149    138.609  -34.9352        52 0 days 10:30:00

[150 rows x 5 columns]


In [4]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [5]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [6]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [7]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [37]:
# raw_preprocessed_df = api_requester.get_solar_forecast()
raw_preprocessed_df, forecast_ready = api_executer.get_new_solar_forecast()

# raw_preprocessed_df = api_requester.get_solar_forecast_cloudmove()
# raw_preprocessed_df, forecast_ready = api_executer.get_new_solar_forecast_cloudmove()
#TODO PRINT FIRST FORECAST CALLED

print(raw_preprocessed_df)

#TODO CHANGE TO 09.30 SE 10.30
#TODO controllare che l'update forecast funziona e che i dati sono veramente diversi

Response status from getforecast: OK.
No new SF forecast
                                    gk  gh_max  gh      tt    ff     rr  \
site_id time                                                              
589020  2023-10-10 22:30:00+09:30  0.0     0.0   0  27.200  14.0  0.025   
        2023-10-10 22:45:00+09:30  0.0     0.0   0  27.125  13.5  0.025   
        2023-10-10 23:00:00+09:30  0.0     0.0   0  27.050  13.0  0.025   
        2023-10-10 23:15:00+09:30  0.0     0.0   0  26.975  12.5  0.025   
        2023-10-10 23:30:00+09:30  0.0     0.0   0  26.900  12.0  0.025   
...                                ...     ...  ..     ...   ...    ...   
589169  2023-10-19 21:30:00+09:30  0.0     0.0   0  16.400  13.0  0.000   
        2023-10-19 21:45:00+09:30  0.0     0.0   0  16.275  13.0  0.000   
        2023-10-19 22:00:00+09:30  0.0     0.0   0  16.150  13.0  0.000   
        2023-10-19 22:15:00+09:30  0.0     0.0   0  16.025  13.0  0.000   
        2023-10-19 22:30:00+09:30  0.0     

In [38]:
sites_df = api_requester.get_current_sites

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
preprocessed_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_preprocessed_df, hours_in_advance)
# preprocessor.save_data2folder()
print(preprocessed_df)

                                        temperature  globalIrradiance  \
cumDistance  time                                                       
1.358064e+03 2023-10-10 22:30:00+09:30       25.200                 0   
             2023-10-10 22:45:00+09:30       25.125                 0   
             2023-10-10 23:00:00+09:30       25.050                 0   
             2023-10-10 23:15:00+09:30       24.975                 0   
             2023-10-10 23:30:00+09:30       24.900                 0   
...                                             ...               ...   
3.026918e+06 2023-10-19 21:30:00+09:30       14.400                 0   
             2023-10-19 21:45:00+09:30       14.275                 0   
             2023-10-19 22:00:00+09:30       14.150                 0   
             2023-10-19 22:15:00+09:30       14.025                 0   
             2023-10-19 22:30:00+09:30       13.900                 0   

                                        windSpeed 

In [10]:
# import folium

# # Assuming you have these dataframes already
# control_stops_df = route.get_control_stops_data
# sites_df = api_requester.get_current_sites
# rt = route.get_route_data

# # Define the bounds
# min_lng = rt['longitude'].min()
# min_lat = rt['latitude'].min()
# max_lng = rt['longitude'].max()
# max_lat = rt['latitude'].max()
# bounds = [[max_lat, min_lng], [min_lat, max_lng]]

# # Create a base map centered around Australia
# middle_lat = (max_lat - min_lat) / 2
# middle_lng = (max_lng - min_lng) / 2
# map = folium.Map(location=[middle_lat, middle_lng],
#                  zoom_start=0)

# # Set the map boundaries to show only Australia
# map.fit_bounds(bounds)

# # Add the route lines
# folium.PolyLine(
#     locations=rt[['latitude', 'longitude']].values.tolist(),
#     color="green",
#     weight=4,
#     opacity=1
# ).add_to(map)

# # Add api sites
# for _, row in sites_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=2,
#         color="blue",
#         fill=True,
#         fill_color="blue",
#         fill_opacity=1,
#         tooltip=row['name']
#     ).add_to(map)

# # Add control stops
# for _, row in control_stops_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=3,
#         color="orange",
#         fill=True,
#         fill_color="orange",
#         fill_opacity=1,
#         tooltip=row['town'] + ': ' + row['location']
#     ).add_to(map)

# folium.Marker(
#                 location=[current_position['latitude'], current_position['longitude']],
#                 icon=folium.Icon(icon="car", prefix="fa")
#         ).add_to(map)

# map.options['maxBounds'] = bounds
# map.options['maxBoundsViscosity'] = 0.9
# map.options['minZoom'] = 5

# map


In [11]:
# from plotter import Plotter

# plotter = Plotter(route)
# plotter.add_api_sites(api_requester.get_current_sites)
# plotter.add_control_stops(route.get_control_stops_data)
# # plotter.add_current_position(current_position)
# current_position = {'longitude': 132.213897,
#                     'latitude': -14.425286}
# plotter.update_max_speed_distance(current_position)
# plotter.plot

# #TODO 
# #SE SI CHIAMA CLOUDMOVE NON ELIMINARE ALTRI DATI METEO MA UNIRLI
# #SE NON CI SONO DATI PLOTTARE NO DATA

In [42]:
#TODO i dati vengono sovrascritti e non aggiornati nei plot
# tenere ultimo SF e aggiornare dati di CM dove possibile
# se non ci sono dati di SF usare CM e viceversa

# se codice runna per tanto, i dropdown del tempo devono essere aggiornati

# per i weather plot bisogna correggere il cumDistance cosi che da 0 fino al primo cumDistance i valori siano 0 per tutti i tempi
# lo stesso vale dall'ultimo punto fino a route_data['cumDistance'].max()
# first_cumDistance = preprocessed_df.index.get_level_values('cumDistance').min()
# last_cumDistance = preprocessed_df.index.get_level_values('cumDistance').max()
# preprocessed_df.loc[(slice
#                     (None, first_cumDistance),
#                     slice(None)), :] = 0
# preprocessed_df.loc[(slice
#                     (last_cumDistance, None),
#                     slice(None)), :] = 0
# print(preprocessed_df)

# range slider per il tempo

# ratio item to take the mean or not of the data in the interval

# cambiare i font

# plot dentro a Div con dimensioni specifiche


from dash import Dash, html, dcc
from dash.dependencies import Input, Output

app = Dash(__name__)

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

## Time
# Slider
vertical_height_slider = 800
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 60

## Map
route_data = route.get_route_data
min_lng = route_data['longitude'].min()
min_lat = route_data['latitude'].min()
max_lng = route_data['longitude'].max()
max_lat = route_data['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_data['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

app.layout = html.Div([
    # Header with image and title
    html.Div([
        # Dropdown
        dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                     value=unique_days[0],
                     id='day-month-year-dropdown',
                     style={'width': '200px'}),

        # Title
        html.H1("Strategy Interface",
                style={'textAlign': 'center', 'margin': '0 auto', 'flex': '1'}),

        # Image
        html.Img(src=url_image,
                 style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
        
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '20px'}),


    # Container for the columns
    html.Div([
        # Time slider
        html.Div(
            dcc.Slider(
                id='hour-minute-slider',
                min=0,
                max=len(unique_days) - 1,
                value=0,
                marks=slider_marks,
                vertical=True,
                step=1,
                included=False,
                verticalHeight=vertical_height_slider
            ), style={'width': '100px', 'height':vertical_height_slider},
            id='hour-minute-slider-container'
        ),

        # Map and static plots
        html.Div(children=[
            dcc.Graph(
                id='map-plot',
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_data['latitude'],
                        'lon': route_data['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'green'},
                        'hoverinfo': 'none'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "open-street-map",
                            'center': {
                                'lat': middle_lat,
                                'lon': middle_lng
                            },
                            'zoom': 3,
                            'uirevision': True
                        },
                        'margin': {"r": 0, "t": 0, "l": 0, "b": 0}
                    }
                }
            ),

            dcc.Graph(
                id='maxSpeed',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['maxSpeed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        # 'title': 'Maximal Allowed Speed',
                        'margin': {
                            "r": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Max Speed (km/h)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        }
                    }
                }
            ),

            dcc.Graph(
                id='inclination',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['inclinationSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        # 'title': 'Inclination Smoothed',
                        'margin': {
                            "r": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Inclination (°)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        }
                    }
                }
            ),
        ], style={'padding': 10, 'flex': 1}),

        # Weather plots
        html.Div(children=[

            dcc.Interval(id='interval-update', interval=update_sec*1000, n_intervals=0),
            html.Div(id='last-update-sf',
                     style={'color': 'gray'}),
            html.Div(id='last-update-cm',
                     style={'color': 'gray'}),

            dcc.Graph(id='plot-globalIrradiance'),

        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    
], style={'display': 'flex', 'flex-direction': 'column'})

## Callbacks
# Last update
@app.callback(
    Output('last-update-sf', 'children'),
    Output('last-update-cm', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is None:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is None:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"
    return print_SF, print_CM

# Time callback
@app.callback(
    [Output('hour-minute-slider', 'max'),
     Output('hour-minute-slider', 'marks'),
     Output('hour-minute-slider-container', 'style')],
     Input('day-month-year-dropdown', 'value')
)
def update_hour_minute_slider(day_month_year):
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times_days)}
    max = len(unique_times_days) - 1
    style = {'width': '100px', 'height': f'{max*60}px'}
    return max, slider_marks, style


@app.callback(
    Output('plot-globalIrradiance', 'figure'),
    [Input('day-month-year-dropdown', 'value'),
    Input('hour-minute-slider', 'value'),
    Input('hour-minute-slider', 'marks')])
def update_graph(day_month_year, hour_minute, slider_marks):
    # Extract timezone from the DataFrame's index
    timezone = preprocessed_df.index.get_level_values('time').tz

    datetime_str = day_month_year + " " + slider_marks[f'{hour_minute}'] # Slider marks are str and hour_minute int
    datetime_obj = pd.Timestamp(datetime_str).tz_localize(timezone)

    data_at_time_df = preprocessed_df.xs(datetime_obj, level='time')
    print(data_at_time_df)

    print(data_at_time_df.index.get_level_values('cumDistance') / 1000)

    figure={
                    'data': [
                        {
                            'x': data_at_time_df.index.get_level_values('cumDistance') / 1000,
                            'y': data_at_time_df['globalIrradiance'],
                            'type': 'scatter',
                            'mode': 'lines'
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Global irradiance (W/m²)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        }
                    }
                }
    return figure

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")

    #TODO SPEGNEER 3D VIEW


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

               temperature  globalIrradiance  windSpeed  windDirection  \
cumDistance                                                              
1358.063809           25.2                 0      1.883          303.0   
21663.838508          24.5                 0      1.480          303.0   
41969.613208          23.4                 0      0.942          335.0   
62275.387907          23.4                 0      0.942          335.0   
82581.162607          24.1                 0      0.538          291.0   
102886.937307         24.0                 0      0.538          291.0   
123192.712006         23.9                 0      0.538          291.0   
143498.486706         24.1                 0      0.673          325.0   
163804.261406         24.8                 0      0.538          325.0   
184110.036105         24.8                 0      0.538          325.0   
204415.810805         25.3                 0      0.807          325.0   
224721.585504         26.4            

In [33]:
# Import necessary libraries
import dash
from dash import dcc, html, dependencies
import plotly.express as px

# Sample data
df1 = px.data.iris()
df2 = px.data.tips()

# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    # First graph
    dcc.Graph(
        id='graph1',
        figure={
            'data': [
                {
                    'x': df1['sepal_width'],
                    'y': df1['sepal_length'],
                    'type': 'scatter',
                    'mode': 'markers',
                    'name': 'Iris Dataset'
                },
            ],
            'layout': {
                'title': 'Iris Dataset',
                'xaxis': {
                    'title': 'Sepal Width',
                    'gridcolor': 'gray',
                    'showgrid': True
                },
                'yaxis': {
                    'title': 'Sepal Length',
                    'gridcolor': 'gray',
                    'showgrid': True
                }
            }
        }
    ),
    
    # Second graph
    dcc.Graph(
        id='graph2',
        figure={
            'data': [
                {
                    'x': df2['total_bill'],
                    'y': df2['tip'],
                    'type': 'scatter',
                    'mode': 'markers',
                    'name': 'Tips Dataset'
                },
            ],
            'layout': {
                'title': 'Tips Dataset',
                'xaxis': {
                    'title': 'Total Bill',
                    'gridcolor': 'gray',
                    'showgrid': True
                },
                'yaxis': {
                    'title': 'Tip',
                    'gridcolor': 'gray',
                    'showgrid': True
                }
            }
        }
    )
])

# @app.callback(
#     dependencies.Output('graph2', 'figure'),
#     dependencies.Input('graph1', 'relayoutData'),
#     dependencies.State('graph2', 'figure')
# )
# def update_graph2_from_graph1(relayout_data, current_figure):
#     if 'xaxis.range[0]' in relayout_data:
#         current_figure['layout']['xaxis']['range'] = [relayout_data['xaxis.range[0]'], relayout_data['xaxis.range[1]']]
#     return current_figure

# @app.callback(
#     dependencies.Output('graph1', 'figure'),
#     dependencies.Input('graph2', 'relayoutData'),
#     dependencies.State('graph1', 'figure')
# )
# def update_graph1_from_graph2(relayout_data, current_figure):
#     if 'xaxis.range[0]' in relayout_data:
#         current_figure['layout']['xaxis']['range'] = [relayout_data['xaxis.range[0]'], relayout_data['xaxis.range[1]']]
#     return current_figure

# Callback to synchronize zoom between the two graphs
@app.callback(
    [dependencies.Output('graph2', 'figure'),
     dependencies.Output('graph1', 'figure')],
    [dependencies.Input('graph1', 'relayoutData'),
     dependencies.Input('graph2', 'relayoutData')],
    [dependencies.State('graph1', 'figure'),
     dependencies.State('graph2', 'figure')]
)
def sync_zoom(relayoutData1, relayoutData2, figure1, figure2):
    # Get the triggering input
    ctx = dash.callback_context
    if not ctx.triggered:
        print("--- no update ---")
        raise dash.exceptions.PreventUpdate
    else:
        input_id = ctx.triggered[0]['prop_id'].split('.')[0]
        print(f"--- {input_id} triggered ---")

    # If zoom, pan, or reset action is taken on graph1, update x-axis of graph2
    if input_id == 'graph1':
        if relayoutData1 and 'autosize' in relayoutData1 and relayoutData1['autosize']:
            print(relayoutData1)
            # Reset x-axis of both graphs to their original view if graph1 was double-clicked
            print(f"Double click on {input_id}")
            return figure2, figure1
        elif relayoutData1 and 'xaxis.range[0]' in relayoutData1:
            updated_figure2 = update_xaxis(figure2, relayoutData1)
            print(f"Update {input_id}")
            return updated_figure2, dash.no_update
        else:
            raise dash.exceptions.PreventUpdate

    # If zoom, pan, or reset action is taken on graph2, update x-axis of graph1
    elif input_id == 'graph2':
        if relayoutData2 and 'autosize' in relayoutData2 and relayoutData2['autosize']:
            # Reset x-axis of both graphs to their original view if graph2 was double-clicked
            print(f"Double click on {input_id}")
            return figure2, figure1
        elif relayoutData2 and 'xaxis.range[0]' in relayoutData2:
            updated_figure1 = update_xaxis(figure1, relayoutData2)
            print(f"Update {input_id}")
            return dash.no_update, updated_figure1
        else:
            raise dash.exceptions.PreventUpdate

def update_xaxis(figure, relayout_data):
    figure['layout']['xaxis']['range'] = [relayout_data['xaxis.range[0]'], relayout_data['xaxis.range[1]']]
    return figure

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

--- no update ---
--- graph1 triggered ---
{'autosize': True}
Double click on graph1
--- no update ---
--- graph1 triggered ---
{'autosize': True}
Double click on graph1
